In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('datatypes').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/06 16:37:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format("csv")\
	.option("header", "true")\
    .option("inferSchema", "true")\
    .load("data/retail-data/by-day/2010-12-01.csv")

In [4]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
df.createOrReplaceTempView("retail")

In [7]:
from pyspark.sql.functions import lit
df.select(lit(5.0),lit("five"),lit(5))

DataFrame[5.0: double, five: string, 5: int]

In [9]:
df1 = spark.sql("select * from retail")

In [10]:
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [11]:
df2 = spark.sql("select * from retail where StockCode = 'DOT' and UnitPrice > 600")
df2.show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      NULL|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [14]:
from pyspark.sql.functions import lit,round,bround
df.select(round(lit(2.5)),bround(lit(2.5))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [15]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
	.withColumn("today", current_date())\
	.withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")
dateDF.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2024-07-06|2024-07-06 17:16:...|
|  1|2024-07-06|2024-07-06 17:16:...|
|  2|2024-07-06|2024-07-06 17:16:...|
|  3|2024-07-06|2024-07-06 17:16:...|
|  4|2024-07-06|2024-07-06 17:16:...|
|  5|2024-07-06|2024-07-06 17:16:...|
|  6|2024-07-06|2024-07-06 17:16:...|
|  7|2024-07-06|2024-07-06 17:16:...|
|  8|2024-07-06|2024-07-06 17:16:...|
|  9|2024-07-06|2024-07-06 17:16:...|
+---+----------+--------------------+



In [17]:
from pyspark.sql.functions import col,date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2024-07-01|        2024-07-11|
+------------------+------------------+
only showing top 1 row



In [18]:
spark